# "Продвинутый Python". Магистерская программа ВШЭ.

### Краснова Дарья мФТиАД181

### Домашнее задание №2

### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [1]:
import json
import datetime #чтобы знать текущий год

In [2]:
PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
    'B5': (250, 176)
}

path_for_rare_editions = "rare_editions.json"

class ReadableEntity:
    '''
    This class creates Journal
    '''
    
    def __init__(self, title, author, genre, number_of_pages, 
                 format_pages, rarity_index, text, year_of_publication):
        
        """
        :param title: title of the ReadableEntity
        :type title: str
        :param author: author of the ReadableEntity
        :type author: str
        :param genre: genre of the ReadableEntity
        :type genre: str
        :param number_of_pages: number of pages of the ReadableEntity (>0)
        :type number_of_pages: int
        :param format_pages: format of pages of the ReadableEntity, from PAGES_FORMAT
        :type format_pages: str
        :param rarity_index: rarity index of the ReadableEntity ([1,10])
        :type rarity_index: int
        :param text: text of the ReadableEntity
        :type text: str

        """
        if not all(isinstance(i, (str)) for i in [title, author, genre,text]):
            raise ValueError("Некорректный формат ввода текстовых полей")
        
        if type(number_of_pages) != int or number_of_pages < 0:
            raise ValueError('Некорректный формат ввода количества страниц.')
              
        if format_pages.upper() not in (PAGES_FORMAT.keys()):
            raise ValueError('Некорректный формат ввода формата.')
        
        if rarity_index not in range(1,11):
            raise ValueError('Некорректный формат ввода индекса редкости.')
            
        if type(year_of_publication) != int  or year_of_publication > datetime.datetime.now().year:
            raise ValueError('Некорректный формат ввода года публикации.')
        
        
        self.title = title
        self.author = author
        self.genre = genre
        self.number_of_pages = number_of_pages
        self.format_pages = format_pages
        self.rarity_index = rarity_index
        self.text = text
        self.year_of_publication = year_of_publication
            
    def get_square(self):
        height, width = PAGES_FORMAT[format_pages.upper()]
        return height * width * self.number_of_pages

    
class Journal(ReadableEntity):
    '''
    This class creates Journal supporting inheritance from ReadableEntity
    
    '''
    def __init__(self, title, author, genre, number_of_pages, format_pages, rarity_index, text, year_of_publication, 
                 issue_number, periodicity):
        '''
        :param issue_number: number of issue in this year
        :type issue_number: int
        :param periodicity: periodicity of journal issue
        :type periodicity: int
        
        :then params of ReadableEntity 

        '''
        if type(issue_number) != int or issue_number < 1:
            raise ValueError('Некорректный формат ввода номера.')
            
        if type(periodicity) != int or periodicity < 1:
            raise ValueError('Некорректный формат ввода периодичности.')
            
        if issue_number > periodicity:
            raise ValueError('Некорректный ввод: номер выпуска формат больше периодичности.')
        
        self.issue_number = issue_number
        self.periodicity = periodicity
        super().__init__(title, author, genre, number_of_pages, format_pages, 
                         rarity_index, text, year_of_publication)
        

class Book(ReadableEntity):
    '''
    This class creates Book supporting inheritance from ReadableEntity
    '''
    def __init__(self, title, author, genre, number_of_pages, format_pages, rarity_index, 
                 text, year_of_publication,
                 publishing_house, book_cover):
        '''
        :param book_cover: type of book cover
        :type book_cover: str
        :param year_of_publication: year of publication of the book
        :type year_of_publication: int
        
        :then params of ReadableEntity 

        '''
        if not all(isinstance(i, (str)) for i in [publishing_house, book_cover]):
            raise ValueError("Некорректный формат ввода текстовых полей")
            
        self.book_cover = book_cover
        
        super().__init__(title, author, genre, number_of_pages, format_pages, 
                         rarity_index, text, year_of_publication)
        

class Exporter:
    '''
    This class creates exporter for rear entities
    '''
    def __init__(self, rarity_of_readable_entity, file_path, save_to_txt = False):
        
        self.rarity_of_readable_entity = rarity_of_readable_entity
        
        if self.rarity_of_readable_entity >= 9:
            if save_to_txt:
                self.export_to_txt(file_path)
            else:
                self.export_to_json(file_path)
   
    def export_to_json(self, file_path):
        with open(file_path, 'a+', encoding='utf-8') as f:
            json.dump(str(self.__dict__), f, ensure_ascii=False)
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key]))       
        print("Saved to txt format")
    

class LibraryEntity:
    '''
    This class creates Library Entity
    
    '''
    def __init__(self, condition_of_entity = None, availability=None, last_borrower=None):
        '''
        :param condition_of_entity: condition of entity
        :type condition_of_entity: str
        :param availability: availability of entity in in library
        :type availability: bool
        :param last_borrower: name of last borrower
        :type last_borrower: str
        '''
        ####
        if condition_of_entity != None and last_borrower != None:
            if not all([isinstance(i, (str))  for i in [condition_of_entity, last_borrower]]):
                raise ValueError("Некорректный формат ввода текстовых полей")
        
        if availability != None:   
            if not isinstance(availability, (bool)):
                raise ValueError('Некорректный формат ввода bool.')
       
        self.condition_of_entity = condition_of_entity
        self.availability = availability
        self.last_borrower = last_borrower
        
class LibraryJournal(Journal, Exporter, LibraryEntity):
        '''
        This class creates Library Journal supporting inheritance from Journal, Exporter, LibraryEntity
        '''
        def __init__(self, title, author, genre, number_of_pages, 
                format_pages, rarity_index, text, year_of_publication, 
                issue_number, periodicity,file_path,
                condition_of_entity = None, availability=None, last_borrower=None,
                save_to_txt = False):

            Journal.__init__(self, title, author, genre, number_of_pages, format_pages, 
                             rarity_index, text, year_of_publication, 
                             issue_number, periodicity)

            LibraryEntity.__init__(self, condition_of_entity, availability, last_borrower)
            
            Exporter.__init__(self, rarity_index, file_path, save_to_txt)


class LibraryBook(Book, Exporter, LibraryEntity):
    '''
        This class creates Library Book supporting inheritance from Book, Exporter, LibraryEntity
    '''
    def __init__(self, title, author, genre, number_of_pages, 
                format_pages, rarity_index, text, year_of_publication, 
                publishing_house, book_cover, file_path,
                condition_of_entity = None, availability=None, last_borrower=None,
                save_to_txt = False):
        
        Book.__init__(self, title, author, genre, number_of_pages, format_pages, 
                      rarity_index, text, year_of_publication, 
                      publishing_house, book_cover)
        LibraryEntity.__init__(self, condition_of_entity, availability, last_borrower)
        Exporter.__init__(self, rarity_index, file_path, save_to_txt)
    

Создадим объект класса - книгу

In [3]:
title = "Гарри Поттер и узник Азкабана"
author = "Дж. К. Роулинг"
genre = "Фэнтези"
number_of_pages = 572
format_pages = "b5"
rarity_index = 10 
text = "Гарри Поттер — необычный мальчик во всех отношениях. Во-первых, он терпеть не может летние каникулы, во-вторых, любит летом делать уроки, но занимается ночью, когда все спят. А самое главное, Гарри Поттер — волшебник. Было уже заполночь. Гарри лежал на животе, с головой укрывшись одеялом. В одной руке — фонарик, а на подушке — старинная толстая книга в кожаном переплете «История магии» Батильды Бэгшот. Сдвинув брови, Гарри водил по строчкам орлиным пером, искал подходящую цитату для сочинения на тему: «Был ли смысл в XIV веке сжигать ведьм?». Перо задержалось на первой строке параграфа. Ага, кажется, то, что нужно. Гарри поправил очки наносу, поднес к странице фонарик и прочитал: В Средние века люди, в чьих жилах нет волшебной крови (более известные как маглы, или простецы), очень боялись колдовства, но отличать настоящих ведьм и колдунов не умели. Иногда им все же удавалось поймать волшебника, но простецы не знали, что волшебникам огонь не страшен: они умели замораживать огонь и притворяться, что им очень больно. На самом же деле они испытывали не боль, а лишь приятное покалывание по всему телу и теплое дуновение воздуха. Так, Венделина Странная очень любила «гореть» на костре. И чтобы испытать это ни с чем не сравнимое удовольствие, сорок семь раз меняла обличье и предавала себя в руки маглов..."
publishing_house = "Росмэн"
book_cover = "Твердый"
year_of_publication = 2001 
file_path = path_for_rare_editions
condition_of_entity = "Хорошее"
availability = None
last_borrower = "Махаон"
save_to_txt = False

In [4]:
harry_book = Book(title, author, genre, number_of_pages, format_pages, rarity_index, text,year_of_publication, \
                 publishing_house, book_cover)

In [5]:
harry_book.__dict__

{'book_cover': 'Твердый',
 'title': 'Гарри Поттер и узник Азкабана',
 'author': 'Дж. К. Роулинг',
 'genre': 'Фэнтези',
 'number_of_pages': 572,
 'format_pages': 'b5',
 'rarity_index': 10,
 'text': 'Гарри Поттер — необычный мальчик во всех отношениях. Во-первых, он терпеть не может летние каникулы, во-вторых, любит летом делать уроки, но занимается ночью, когда все спят. А самое главное, Гарри Поттер — волшебник. Было уже заполночь. Гарри лежал на животе, с головой укрывшись одеялом. В одной руке — фонарик, а на подушке — старинная толстая книга в кожаном переплете «История магии» Батильды Бэгшот. Сдвинув брови, Гарри водил по строчкам орлиным пером, искал подходящую цитату для сочинения на тему: «Был ли смысл в XIV веке сжигать ведьм?». Перо задержалось на первой строке параграфа. Ага, кажется, то, что нужно. Гарри поправил очки наносу, поднес к странице фонарик и прочитал: В Средние века люди, в чьих жилах нет волшебной крови (более известные как маглы, или простецы), очень боялись ко

In [6]:
harry_library_book = LibraryBook(title, author, genre, number_of_pages, 
                format_pages, rarity_index, text, year_of_publication,
                publishing_house, book_cover, file_path,
                condition_of_entity, availability, last_borrower,
                save_to_txt)

Теперь, библиотечную книгу

In [7]:
harry_library_book.__dict__

{'book_cover': 'Твердый',
 'title': 'Гарри Поттер и узник Азкабана',
 'author': 'Дж. К. Роулинг',
 'genre': 'Фэнтези',
 'number_of_pages': 572,
 'format_pages': 'b5',
 'rarity_index': 10,
 'text': 'Гарри Поттер — необычный мальчик во всех отношениях. Во-первых, он терпеть не может летние каникулы, во-вторых, любит летом делать уроки, но занимается ночью, когда все спят. А самое главное, Гарри Поттер — волшебник. Было уже заполночь. Гарри лежал на животе, с головой укрывшись одеялом. В одной руке — фонарик, а на подушке — старинная толстая книга в кожаном переплете «История магии» Батильды Бэгшот. Сдвинув брови, Гарри водил по строчкам орлиным пером, искал подходящую цитату для сочинения на тему: «Был ли смысл в XIV веке сжигать ведьм?». Перо задержалось на первой строке параграфа. Ага, кажется, то, что нужно. Гарри поправил очки наносу, поднес к странице фонарик и прочитал: В Средние века люди, в чьих жилах нет волшебной крови (более известные как маглы, или простецы), очень боялись ко

Площадь занимает издание, если разложить все его страницы на полу

In [8]:
harry_library_book.get_square()

25168000

Проверим, записалось ли все в файл

In [9]:
with open(path_for_rare_editions, "r", encoding="utf-8") as r:
    print(r.read())

"{'book_cover': 'Твердый', 'title': 'Гарри Поттер и узник Азкабана', 'author': 'Дж. К. Роулинг', 'genre': 'Фэнтези', 'number_of_pages': 572, 'format_pages': 'b5', 'rarity_index': 10, 'text': 'Гарри Поттер — необычный мальчик во всех отношениях. Во-первых, он терпеть не может летние каникулы, во-вторых, любит летом делать уроки, но занимается ночью, когда все спят. А самое главное, Гарри Поттер — волшебник. Было уже заполночь. Гарри лежал на животе, с головой укрывшись одеялом. В одной руке — фонарик, а на подушке — старинная толстая книга в кожаном переплете «История магии» Батильды Бэгшот. Сдвинув брови, Гарри водил по строчкам орлиным пером, искал подходящую цитату для сочинения на тему: «Был ли смысл в XIV веке сжигать ведьм?». Перо задержалось на первой строке параграфа. Ага, кажется, то, что нужно. Гарри поправил очки наносу, поднес к странице фонарик и прочитал: В Средние века люди, в чьих жилах нет волшебной крови (более известные как маглы, или простецы), очень боялись колдовст

Теперь, проделаем все то же самое с журналом

In [10]:
title = "Esquire"
author = "Сергей Минаев"
genre = "Бизнес"
number_of_pages = 150
format_pages = "a4"
rarity_index = 3 
text = "Запуск мартовского номера Esquire c главным материалом года, в котором редакция выбирает новых героев поколения из 12 разных областей — людей, которые сегодня становятся лидерами в режиссуре и литературе, в бизнесе и технологиях, в музыке и гастрономии..."
issue_number = 2
periodicity = 11
year_of_publication = 2019 
file_path = path_for_rare_editions
condition_of_entity = "Плохое"
availability = True
last_borrower = None
save_to_txt = False


In [11]:
esquire_journ = Journal(title, author, genre, number_of_pages, format_pages, rarity_index, text, year_of_publication, 
                 issue_number, periodicity)

In [12]:
esquire_journ.__dict__

{'issue_number': 2,
 'periodicity': 11,
 'title': 'Esquire',
 'author': 'Сергей Минаев',
 'genre': 'Бизнес',
 'number_of_pages': 150,
 'format_pages': 'a4',
 'rarity_index': 3,
 'text': 'Запуск мартовского номера Esquire c главным материалом года, в котором редакция выбирает новых героев поколения из 12 разных областей — людей, которые сегодня становятся лидерами в режиссуре и литературе, в бизнесе и технологиях, в музыке и гастрономии...',
 'year_of_publication': 2019}

In [13]:
esquire_journ.get_square(), esquire_journ.number_of_pages * 297 * 210

(9355500, 9355500)

In [14]:
esquire_library_journ = LibraryJournal(title, author, genre, number_of_pages, 
                format_pages, rarity_index, text, year_of_publication, 
                issue_number, periodicity,file_path,
                condition_of_entity, availability, last_borrower,
                save_to_txt)

In [15]:
esquire_library_journ.__dict__

{'issue_number': 2,
 'periodicity': 11,
 'title': 'Esquire',
 'author': 'Сергей Минаев',
 'genre': 'Бизнес',
 'number_of_pages': 150,
 'format_pages': 'a4',
 'rarity_index': 3,
 'text': 'Запуск мартовского номера Esquire c главным материалом года, в котором редакция выбирает новых героев поколения из 12 разных областей — людей, которые сегодня становятся лидерами в режиссуре и литературе, в бизнесе и технологиях, в музыке и гастрономии...',
 'year_of_publication': 2019,
 'condition_of_entity': 'Плохое',
 'availability': True,
 'last_borrower': None,
 'rarity_of_readable_entity': 3}

In [16]:
esquire_library_journ.get_square()


9355500

Журнал не записался в json, потому что издание - не редкое

In [17]:
with open(path_for_rare_editions, "r", encoding="utf-8") as r:
    print(r.read())

"{'book_cover': 'Твердый', 'title': 'Гарри Поттер и узник Азкабана', 'author': 'Дж. К. Роулинг', 'genre': 'Фэнтези', 'number_of_pages': 572, 'format_pages': 'b5', 'rarity_index': 10, 'text': 'Гарри Поттер — необычный мальчик во всех отношениях. Во-первых, он терпеть не может летние каникулы, во-вторых, любит летом делать уроки, но занимается ночью, когда все спят. А самое главное, Гарри Поттер — волшебник. Было уже заполночь. Гарри лежал на животе, с головой укрывшись одеялом. В одной руке — фонарик, а на подушке — старинная толстая книга в кожаном переплете «История магии» Батильды Бэгшот. Сдвинув брови, Гарри водил по строчкам орлиным пером, искал подходящую цитату для сочинения на тему: «Был ли смысл в XIV веке сжигать ведьм?». Перо задержалось на первой строке параграфа. Ага, кажется, то, что нужно. Гарри поправил очки наносу, поднес к странице фонарик и прочитал: В Средние века люди, в чьих жилах нет волшебной крови (более известные как маглы, или простецы), очень боялись колдовст

Проверка формата ввода

In [18]:
format_pages = "a_4"
esquire_library_journ = LibraryJournal(title, author, genre, number_of_pages, 
                format_pages, rarity_index, text, year_of_publication, 
                issue_number, periodicity,file_path,
                condition_of_entity, availability, last_borrower,
                save_to_txt)

ValueError: Некорректный формат ввода формата.

Поменяем коэффициент редкости

In [19]:
format_pages = "a4"
rarity_index = 9
esquire_library_journ = LibraryJournal(title, author, genre, number_of_pages, 
                format_pages, rarity_index, text, year_of_publication, 
                issue_number, periodicity,file_path,
                condition_of_entity, availability, last_borrower,
                save_to_txt)

with open(path_for_rare_editions, "r", encoding="utf-8") as r:
    print(r.read())

"{'book_cover': 'Твердый', 'title': 'Гарри Поттер и узник Азкабана', 'author': 'Дж. К. Роулинг', 'genre': 'Фэнтези', 'number_of_pages': 572, 'format_pages': 'b5', 'rarity_index': 10, 'text': 'Гарри Поттер — необычный мальчик во всех отношениях. Во-первых, он терпеть не может летние каникулы, во-вторых, любит летом делать уроки, но занимается ночью, когда все спят. А самое главное, Гарри Поттер — волшебник. Было уже заполночь. Гарри лежал на животе, с головой укрывшись одеялом. В одной руке — фонарик, а на подушке — старинная толстая книга в кожаном переплете «История магии» Батильды Бэгшот. Сдвинув брови, Гарри водил по строчкам орлиным пером, искал подходящую цитату для сочинения на тему: «Был ли смысл в XIV веке сжигать ведьм?». Перо задержалось на первой строке параграфа. Ага, кажется, то, что нужно. Гарри поправил очки наносу, поднес к странице фонарик и прочитал: В Средние века люди, в чьих жилах нет волшебной крови (более известные как маглы, или простецы), очень боялись колдовст

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [20]:
import sys

Есть такой [вариант](https://goshippo.com/blog/measure-real-size-any-python-object/).

In [21]:
def get_real_size_of_objects(object_to_measure, ids_of_seen_obj=None):    
    """
        This function recursively finds the size of objects
        
        :param object_to_measure: the object which size we want to find
        :type object_to_measure: any
        :param ids_of_seen_obj: list of seen ids
        :type ids_of_seen_obj: list
        
        
        :returns: object size in bytes
        :rtype: int
    """
    size_of_obj = sys.getsizeof(object_to_measure)
    
    if ids_of_seen_obj is None:
        ids_of_seen_obj = []
    
    object_id = id(object_to_measure)
    
    if object_id in ids_of_seen_obj:
        return 0
    else:
        ids_of_seen_obj.append(object_id)
        size_of_obj = sys.getsizeof(object_to_measure)
        
        if isinstance(object_to_measure, (int, float, complex, str)):
            return size_of_obj
        
        elif type(object_to_measure) ==  dict:
            size_of_obj += sum([get_real_size_of_objects(v, ids_of_seen_obj) for v in object_to_measure.values()])
            size_of_obj += sum([get_real_size_of_objects(k, ids_of_seen_obj) for k in object_to_measure.keys()])
            return size_of_obj
        
        elif isinstance(object_to_measure, (list, tuple)):
            size_of_obj += sum([get_real_size_of_objects(v, ids_of_seen_obj) for v in object_to_measure])
            return size_of_obj
        
        ##продвинутости для класса
        elif hasattr(object_to_measure, '__dict__'):
            #например, словарь объекта класса содержит аттрибут и принимаемые значения аттрибута
            #функция итеративно рекурсией пройдет по всем атрибутам и запишет размер
            size_of_obj += get_real_size_of_objects(object_to_measure.__dict__, ids_of_seen_obj)
            return size_of_obj
        
        elif hasattr(object_to_measure, '__iter__'):
            #так же для итераторов
            size_of_obj += sum([get_real_size_of_objects(i, ids_of_seen_obj) for i in object_to_measure])
            return size_of_obj
        
        else:
            raise ValueError("couldn't get size")



In [22]:
get_real_size_of_objects(1+2j)

32

In [23]:
get_real_size_of_objects(([])), get_real_size_of_objects(['aaaaaaa']), get_real_size_of_objects('aaaaaaa')

(64, 128, 56)

In [24]:
get_real_size_of_objects(range(1,9,2))

160

In [25]:
get_real_size_of_objects(harry_book)

3992

### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

- Для быстрого умножения использовалось [быстрое преобразование Фурье](https://e-maxx.ru/algo/fft_multiply)
- Для быстрого возведения в степень использовалось [бинарное возведение в степень](https://e-maxx.ru/algo/binary_pow)

In [26]:
from cmath import exp, pi
def fast_fourier_transform(coeffs, Inverse_flag = False):
    """
        This function finds the fast Fourier transform
        
        :param coeffs: coefficients of the polynom
        :type coeffs: list
        :param Inverse_flag: flag of inverse coefficients
        :type Inverse_flag: bool
        
        :returns: fft coefficients
        :rtype: list
    """
    if Inverse_flag:
        coeffs = [(c.imag + 1j * c.real) for c in coeffs]
        
    n = len(coeffs)
    if n <= 1:
        return [sum(coeffs)]

    a1 = fast_fourier_transform(coeffs[0::2])
    a2 = fast_fourier_transform(coeffs[1::2])
    
    w = [exp(-2j * pi * k / n) * a2[k] for k in range(n // 2)]
    
    y1 = [a1[k] + w[k] for k in range(n//2)]
    y2 = [a1[k] - w[k] for k in range(n//2)]
    
    return y1+y2

In [27]:
class Polynom:
    
    def __init__(self, coeffs):
        """
        input: a_0, a_1, a_2, ...a_n
        """
        """
        :param coeffs: coefficients of polynom with such order
            a_0, a_1, a_2, ...a_n 
            (a_0 + a_1 * x + a_2 * x^2 +...+ a_n * x^n)
        :type coeffs: list/int/complex

        """
        
        self.coeffs = self.get_coeffs(coeffs)
        if len(self.coeffs) == 0:
            self.coeffs = [0]
        self.degree = len(self.coeffs) - 1
        
    def check_input(self, coeffs):
        if not all(isinstance(i, (int,float,complex)) for i in coeffs):
            raise ValueError("Wrong data type for polynom coefficients")
        return coeffs
            
    def get_coeffs(self, coeffs):
        if type(coeffs) != list:
            return self.check_input([coeffs])
        for c in coeffs[::-1]:
            if c == 0:
                coeffs.pop()
            else:
                break
        return self.check_input(coeffs)
    
    def broadcast_polynom(self, another_polynom):
        polynom_1_len_coeffs = len(self.coeffs)
        polynom_2_len_coeffs = len(another_polynom.coeffs)
        
        if polynom_1_len_coeffs > polynom_2_len_coeffs:
            coeffs_2 = [0] * polynom_1_len_coeffs
            coeffs_2[:polynom_2_len_coeffs] = another_polynom.coeffs
            return self.coeffs, coeffs_2
        else:
            coeffs_1 = [0] * polynom_2_len_coeffs
            coeffs_1[:polynom_1_len_coeffs] = self.coeffs
            return coeffs_1, another_polynom.coeffs
    
        
    #1. Сложение 
    def __add__(self, another_polynom):
        if type(another_polynom) != type(self):
            another_polynom = Polynom(another_polynom)
        coeffs_1, coeffs_2 = self.broadcast_polynom(another_polynom)
        new_coeffs = [i + j for i,j in zip(coeffs_1, coeffs_2)]
        return Polynom(new_coeffs)
    
    
    #2. Вычитание 
    def __sub__(self, another_polynom):
        if type(another_polynom) != type(self):
            another_polynom = Polynom(another_polynom)
        coeffs_1, coeffs_2 = self.broadcast_polynom(another_polynom)
        new_coeffs = [i - j for i,j in zip(coeffs_1, coeffs_2)]
        return Polynom(new_coeffs)
    
    
    #3. Умножение
    def __mul__(self, another_polynom):
        if type(another_polynom) != type(self):
            another_polynom = Polynom(another_polynom)
        coeffs_1, coeffs_2 = self.broadcast_polynom(another_polynom)
        polynom_1_len_coeffs, polynom_2_len_coeffs = len(coeffs_1), len(coeffs_2)
        
        new_coeffs = [0] * (polynom_1_len_coeffs + polynom_2_len_coeffs)
        for i in range(polynom_1_len_coeffs):
            for j in range(polynom_2_len_coeffs):
                new_coeffs[i + j] += coeffs_1[i] * coeffs_2[j]

        return Polynom(new_coeffs)
    
    #3а. Быстрое умножение
    def __fast_mul__(self, another_polynom): 
        if type(another_polynom) != type(self):
            another_polynom = Polynom(another_polynom)
            
        coeffs_1, coeffs_2 = self.broadcast_polynom(another_polynom)
        n = 2**len(coeffs_1)

        while len(coeffs_1) != n: 
            coeffs_1.append(0)
        while len(coeffs_2) != n: 
            coeffs_2.append(0)

        coeff_1 = fast_fourier_transform(coeffs_1)
        coeff_2 = fast_fourier_transform(coeffs_2)
        
        new_coeff = [coeff_1[i] * coeff_2[i] for i in range(n)]
        
        new_coeffs = fast_fourier_transform(new_coeff, True)
        
        new_coeffs = [round(a.imag / n) for a in new_coeffs] 
        new_coeffs = self.get_coeffs(new_coeffs)
            
        return Polynom(new_coeffs)
    
    
    #4. Деление
    #4а. Деление на число
    def __devision_by_number__(self, number):
        if number == 0:
            raise ValueError("Devision by zero")
        another_polynom = Polynom(1 / number)
        return self.__mul__(another_polynom)
    
    
    #5. Возведение в степень 
    def __pow__(self, pow_):
        powed_polynom = Polynom(1)
        for _ in range(pow_):
            powed_polynom *= self
        return powed_polynom
    
    # 5a. быстрое возведение в степень
    def __fast_pow__(self, pow_):
        powed_polynom = self.fast_recursive_pow(pow_)
        return powed_polynom
        
    def fast_recursive_pow(self, pow_):
        if pow_ > 0:
            if pow_ % 2:
                return self.fast_recursive_pow(pow_ - 1) * self
            else:
                return (self * self).fast_recursive_pow(pow_ / 2)
        else:
            return Polynom(1)
    
    
    #6. Представление многочлена в человеческом виде
    def __str__(self):
        polynom_good_representation = ''
        
        for iteration_power, coeff in enumerate(self.coeffs[::-1]):
            current_power = self.degree - iteration_power
            if coeff != 0:
                condition_for_ones = (abs(coeff == 1) and current_power != 0)
                if coeff.imag == 0:
                    coeff = coeff.real
                if type(coeff) == complex:
                    if current_power == self.degree :
                        polynom_good_representation += str(coeff)
                    else:
                        polynom_good_representation += '+' + str(coeff)                 
                else:
                    if coeff > 0:
                        if current_power == self.degree:
                            polynom_good_representation += condition_for_ones * str() + (not condition_for_ones) * str(coeff)
                        else:
                            polynom_good_representation += '+' + condition_for_ones * str() + (not condition_for_ones) * str(coeff)
                    else:
                        polynom_good_representation += condition_for_ones * '-' + (not condition_for_ones) * str(coeff)

                if current_power != 0:
                    polynom_good_representation += 'x'  
                    if current_power != 1:
                        for digit in str(current_power):
                            if digit == '1':
                                 #без eval не получ пока что
                                polynom_good_representation += eval(r'"\u00b{}"'.format(str(9))) 
                            elif digit < '4' and digit != '0':
                                polynom_good_representation += eval(r'"\u00b{}"'.format(str(digit))) 
                            else:
                                polynom_good_representation += eval(r'"\u207{}"'.format(str(digit))) 
            elif self.degree == 0:
                polynom_good_representation += str(coeff)
            
        return polynom_good_representation

    
    #7. Дифференцирование
    def find_derivative(self):
        if len(self.coeffs) == 1:
            return Polynom(0)
        else:
            derivative = [i * c for i, c in enumerate(self.coeffs)]
        return Polynom(derivative[1:])
    
    
    #8. Интегрирование
    def find_integral(self):
        integrals = [coeff / (step + 1) for step, coeff in enumerate(self.coeffs)]
        return Polynom([0] + integrals)
     
    
    #9. Вызов многочлена как функции (вычисление значения в точке)
    def __call__(self, x):    
        call_polynom_in_x = self.coeffs[0]
        for i, coeff in enumerate(self.coeffs[1:]):
            call_polynom_in_x += coeff * x**i
        return call_polynom_in_x 

Проверим обработку ввода

In [28]:
Polynom([1,1,1,1,9,1,1,1,1,1,'ty',1+2j])

ValueError: Wrong data type for polynom coefficients

In [29]:
print('Polynoms with real coefficients:')
p1 = Polynom([0, 0, 5])
print('p1 =', p1)

p2 = Polynom([1,-2, 1, 0])
print('p2 =', p2)

p3 = Polynom([0, 2, 0, 4])
print('p3 =', p3)

print('\n')

print('Polynoms with complex coefficients::')
q1 = Polynom([2+8j, -5, 3, 8])
print('q1 =', q1)

q2 = Polynom([2+8j, -5, 0, 8])
print('q2 =', q2)

q3 = Polynom([0, 10+5j])
print('q3 =', q3)

Polynoms with real coefficients:
p1 = 5x²
p2 = x²-2x+1
p3 = 4x³+2x


Polynoms with complex coefficients::
q1 = 8x³+3x²-5x+(2+8j)
q2 = 8x³-5x+(2+8j)
q3 = (10+5j)x


In [30]:
#1. Сложение
print('--')
print('p1 + p2 =', p1 + p2)
print('p3 + 8 =', p3 + 8)

print('--')
print('q1 + q2 =', q1 + q2)
print('q1 + 2 =', q1 + 2)
print('--')

--
p1 + p2 = 6x²-2x+1
p3 + 8 = 4x³+2x+8
--
q1 + q2 = 16x³+3x²-10x+(4+16j)
q1 + 2 = 8x³+3x²-5x+(4+8j)
--


In [31]:
#2. Вычитание
print('--')
print('p1 - p2 =', p1 - p2)
print('p3 - 8 =', p3 - 8)

print('--')
print('q1 - q2 =', q1 - q2)
print('q1 - 2 =', q1 - 2)
print('--')

--
p1 - p2 = 4x²+2x-1
p3 - 8 = 4x³+2x-8
--
q1 - q2 = 3x²
q1 - 2 = 8x³+3x²-5x+8j
--


In [32]:
#3. Умножение
print('--')
print('p1 * p2 =', p1 * p2)
print('p3 * 8 =', p3 * 8)

print('--')
print('q1 * q2 =', q1 * q2)
print('q1 * 2 =', q1 * 2)
print('--')

--
p1 * p2 = 5x⁴-10x³+5x²
p3 * 8 = 32x³+16x
--
q1 * q2 = 64x⁶+24x⁵-80x⁴+(17+128j)x³+(31+24j)x²+(-20-80j)x+(-60+32j)
q1 * 2 = 16.0x³+6.0x²-10.0x+(4+16j)
--


In [33]:
#3а. Быстрое умножение
print('--')
print('p1 * p2 =', p1.__fast_mul__(p2))
print('p3 * 8 =', p1.__fast_mul__(8))

print('--')


--
p1 * p2 = 5x⁴-10x³+5x²
p3 * 8 = 40x²
--


In [34]:
import time

In [35]:
t1 = Polynom([5]*10)
t2 = Polynom([5]*10)

In [36]:
start = time.time()
print(t1.__fast_mul__(t2))
time.time() - start

25x¹⁸+50x¹⁷+75x¹⁶+100x¹⁵+125x¹⁴+150x¹³+175x¹²+200x¹¹+225x¹⁰+250x⁹+225x⁸+200x⁷+175x⁶+150x⁵+125x⁴+100x³+75x²+50x+25


0.017522096633911133

In [37]:
start = time.time()
print(t1*t2)
time.time() - start

25x¹⁸+50x¹⁷+75x¹⁶+100x¹⁵+125x¹⁴+150x¹³+175x¹²+200x¹¹+225x¹⁰+250x⁹+225x⁸+200x⁷+175x⁶+150x⁵+125x⁴+100x³+75x²+50x+25


0.16398906707763672

In [38]:
#4а. Деление на число
print('--')
print('p1 / 5 =', p1.__devision_by_number__(5))

print('--')
print('q1 / 2 =', q1.__devision_by_number__(2))
print('--')

--
p1 / 5 = x²
--
q1 / 2 = 4.0x³+1.5x²-2.5x+(1+4j)
--


In [39]:
#5. Возведение в степень
print('--')
print('p1 ** 2 =', p1**2)

print('--')
print('q3 ** 2 =', q3**2)
print('--')

--
p1 ** 2 = 25x⁴
--
q3 ** 2 = (75+100j)x²
--


In [40]:
#5а. Быстрое возведение в степень
print('--')
print('p1 ** 2 =', p1.__fast_pow__(2))

print('--')
print('q3 ** 2 =', q3.__fast_pow__(2))
print('--')

--
p1 ** 2 = 25x⁴
--
q3 ** 2 = (75+100j)x²
--


Сравним время работы

In [41]:
start = time.time()
t1.__fast_pow__(25)
time.time() - start

0.016499042510986328

In [42]:
start = time.time()
t1**25
time.time() - start

0.06755208969116211

In [43]:
#6. Представление многочлена в человеческом виде
print(Polynom([5] + [2]*100))

2x¹⁰⁰+2x⁹⁹+2x⁹⁸+2x⁹⁷+2x⁹⁶+2x⁹⁵+2x⁹⁴+2x⁹³+2x⁹²+2x⁹¹+2x⁹⁰+2x⁸⁹+2x⁸⁸+2x⁸⁷+2x⁸⁶+2x⁸⁵+2x⁸⁴+2x⁸³+2x⁸²+2x⁸¹+2x⁸⁰+2x⁷⁹+2x⁷⁸+2x⁷⁷+2x⁷⁶+2x⁷⁵+2x⁷⁴+2x⁷³+2x⁷²+2x⁷¹+2x⁷⁰+2x⁶⁹+2x⁶⁸+2x⁶⁷+2x⁶⁶+2x⁶⁵+2x⁶⁴+2x⁶³+2x⁶²+2x⁶¹+2x⁶⁰+2x⁵⁹+2x⁵⁸+2x⁵⁷+2x⁵⁶+2x⁵⁵+2x⁵⁴+2x⁵³+2x⁵²+2x⁵¹+2x⁵⁰+2x⁴⁹+2x⁴⁸+2x⁴⁷+2x⁴⁶+2x⁴⁵+2x⁴⁴+2x⁴³+2x⁴²+2x⁴¹+2x⁴⁰+2x³⁹+2x³⁸+2x³⁷+2x³⁶+2x³⁵+2x³⁴+2x³³+2x³²+2x³¹+2x³⁰+2x²⁹+2x²⁸+2x²⁷+2x²⁶+2x²⁵+2x²⁴+2x²³+2x²²+2x²¹+2x²⁰+2x¹⁹+2x¹⁸+2x¹⁷+2x¹⁶+2x¹⁵+2x¹⁴+2x¹³+2x¹²+2x¹¹+2x¹⁰+2x⁹+2x⁸+2x⁷+2x⁶+2x⁵+2x⁴+2x³+2x²+2x+5


In [44]:
#7. Дифференцирование
print('--')
print('(p1)\' =', p1.find_derivative())

print('--')
print('(q1)\'=', q1.find_derivative())
print('--')

--
(p1)' = 10x
--
(q1)'= 24x²+6x-5
--


In [45]:
#8. Интегрирование
print('--')
print('int(p3) =', p3.find_integral())

print('--')
print('int(q3)=', q3.find_integral())
print('--')

--
int(p3) = x⁴+x²
--
int(q3)= (5+2.5j)x²
--


In [46]:
#9. Вызов многочлена как функции (вычисление значения в точке)
print('--')
print('p3(5) =', p3(5))

print('--')
print('q3(1) =', q3(5))
print('--')

--
p3(5) = 102
--
q3(1) = (10+5j)
--


In [47]:
eval(r'"\u00b' + str(9) + '"') 

'¹'

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [48]:
print(range.__doc__)

range(stop) -> range object
range(start, stop[, step]) -> range object

Return an object that produces a sequence of integers from start (inclusive)
to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
These are exactly the valid indices for a list of 4 elements.
When step is given, it specifies the increment (or decrement).


In [49]:
def _range(*args):
    """
        This function generates an iterator with the same behavior as range
        Return a sequence of integers from start (inclusive)
        to stop (exclusive) by step. 
        
        _range(stop) -> 0, 1, .. stop-1
        _range(start, stop) -> start, start+1, .. stop-1
        _range(start, stop, step) -> start, start+step, .. stop-step
        
        :param start: start of sequence (inclusive)
        :type start: int
        :param start: stop of sequence (exclusive)
        :type start: int
        :param step: step for sequence
        :type start: int
        
        :returns: iterator sequence
        :rtype: list
    """
    
    if len(args) > 3:
        raise TypeError("range expected at most 3 arguments")
    if not all(isinstance(i, int) for i in args):
        raise ValueError("Wrong data type for _range")

    if len(args) == 1:
        start, stop, step = 0, *args, 1 
    elif len(args) == 2:
        start, stop, step = *args, 1
    else:
        start, stop, step = args
    
    iteration_step = start
    
    if step > 0:
        while iteration_step < stop:
            yield iteration_step
            iteration_step += step
    else:
        while iteration_step > stop:
            yield iteration_step
            iteration_step += step


In [50]:
list(_range(4,0,-1)), list(_range(0,4,1)), list(_range(0)), list(_range(5))

([4, 3, 2, 1], [0, 1, 2, 3], [], [0, 1, 2, 3, 4])

In [51]:
list(range(4,0,-1)), list(range(0,4,1)), list(range(0)), list(range(5))

([4, 3, 2, 1], [0, 1, 2, 3], [], [0, 1, 2, 3, 4])

Рассмотрим обработку возможных ошибок ввода

In [52]:
list(_range('4',0,-1))

ValueError: Wrong data type for _range

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

Реализуем класс, который будет создавать запись таблицы с primary key и другим полем (пусть, в нашем пример это будет текст) 
PrimaryKey может быть составным, для этого будем передавать лист в primary key.

In [53]:
class DescriptorPrimaryKey: 
    '''
    This class is descriptor for field with properties of Primary Key
    
    '''
    def __init__(self, name = None): 
        self.name = name 
        self.primary_keys = [] 
        self.primary_keys_amount = None 

    def __get__(self, instance, owner): 
        return instance.__dict__[self.name] 

    def __set__(self, instance, key): 
        '''
        :param instance: instance of the descriptor owner class
        :type instance: CreateTableRecord.primary_key
                        
        :param key: value that is assigned to primary key 
        :type instance: int/str/  OR list of int/str/ (!= None)
                        PrimaryKey unique for all valus
                        For primary keys with multiple fields check that the amound of fields equal
        '''
    
        if None in key: 
            raise ValueError("Any part of PrimaryKey must not be equall to Null") 

        if all(type(i) == int for i in key) or all(type(i) == str for i in [key]): 

            if len(self.primary_keys) == 0: 
                self.primary_keys_amount = len(key) 

            elif len(key) != self.primary_keys_amount: 
                raise ValueError("All PrimaryKeys must contain the same amount of fields") 

            if key not in self.primary_keys: 
                instance.__dict__[self.name] = key 
                self.primary_keys.append(key) 
            else: 
                raise ValueError("PrimaryKey must be unique") 
        else: 
            raise ValueError("Wrong type for PrimaryKey") 
       
    def __delete__(self, instance): 
        #remove primary key for deleted instance from our container 
        if instance.__dict__[self.name] in self.primary_keys:
            self.primary_keys.remove(instance.__dict__[self.name]) 
        if len(self.primary_keys) == 0:
            self.primary_keys_amount = None 
        
class CreateTableRecord: 
    '''
    This class creates table record with primary key 
    '''
    primary_key = DescriptorPrimaryKey('primary_key') 

    def __init__(self, primary_key, other_field): 
        '''
            :param primary_key: value(s) that is assigned to primary key 
            :type primary_key: int/str/  OR list of int/str/ (!= None)
                            PrimaryKey unique for all valus
                            For primary keys with multiple fields check that the amound of fields equal

            :param other_field: value that is assigned to the other field
            :type other_field: str (or whatever)
        '''
        
        if type(primary_key) != list: 
            self.primary_key = [primary_key] 
        else: 
            self.primary_key = primary_key 

        self.other_field = other_field 

    def __del__(self): 
        del self.primary_key 
        del self

Рассмотрим обработку возможных ошибок ввода и правильность работы класса (в частности удаления и проверку ограничений primary key)

In [54]:
y1 = CreateTableRecord([4,4], "some text")

In [55]:
print(y1.primary_key, y1.other_field)

[4, 4] some text


In [56]:
y2 = CreateTableRecord([4,2], "some text 2")

In [57]:
print(y2.primary_key, y2.other_field)

[4, 2] some text 2


In [58]:
y3 = CreateTableRecord([4,2], "mustn't be written")

ValueError: PrimaryKey must be unique

In [59]:
del y2

In [60]:
y3 = CreateTableRecord([4,2], "now, it's ok")

In [61]:
y4 = CreateTableRecord([4], "error for amount of key fields")

ValueError: All PrimaryKeys must contain the same amount of fields

In [62]:
y4 = CreateTableRecord([4,None], "error: Null key")

ValueError: Any part of PrimaryKey must not be equall to Null

In [63]:
del y3

In [64]:
del y1

Поскольку удалили все предыдущие объекты, тперь можем использовать 1 поле в качестве primary key

In [65]:
x1 = CreateTableRecord(1, "ok")

In [66]:
x2 = CreateTableRecord(2, "ok 2")

In [67]:
x3 = CreateTableRecord(2, "not ok: mustn't be written")

ValueError: PrimaryKey must be unique

In [68]:
del x2

In [69]:
x3 = CreateTableRecord(2, "now it's ok")

In [70]:
print(x1.primary_key, x1.other_field)

[1] ok


### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

По метериалам лекции 3

In [71]:
class DescriptorPositiveSmallIntegerField:
    '''
    This class is descriptor for field with type PositiveSmallIntegerField
    '''
    def __init__(self, name = None):
        self.name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if type(value) == int and 0 <= value <= 32767:
            instance.__dict__[self.name] = value
        else:
            raise ValueError("PositiveSmallIntegerField принимает целые значения в диапазоне:[0, 32767]. Проверьте ввод")
                
    def __delete__(self, object_):
        del instance


In [72]:
class PositiveSmallIntegerField:
    '''
    This class defines field with type PositiveSmallIntegerField
    '''
    value = DescriptorPositiveSmallIntegerField("positive_small_int")

    def __init__(self, number):
        self.value = number

In [73]:
PositiveSmallIntegerField(32767).value

32767

Рассмотрим обработку возможных ошибок ввода

In [74]:
PositiveSmallIntegerField(32769)

ValueError: PositiveSmallIntegerField принимает целые значения в диапазоне:[0, 32767]. Проверьте ввод

In [75]:
PositiveSmallIntegerField('5')

ValueError: PositiveSmallIntegerField принимает целые значения в диапазоне:[0, 32767]. Проверьте ввод

In [77]:
PositiveSmallIntegerField(-5)

ValueError: PositiveSmallIntegerField принимает целые значения в диапазоне:[0, 32767]. Проверьте ввод

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

Данный пример хорошо разбирается [тут](https://www.coursera.org/lecture/diving-in-python/kontiekstnyie-mieniedzhiery-CXVes)

In [78]:
import time
class TimerContextManager:
    '''
    This class calculates the time that was spent in it
    '''
    def __init__(self):
        self.init_time = time.time()

    def __enter__(self):
        return("Hey! I'm context manager!")

    def __exit__(self, type, value, traceback):
        print(f"The time that was spent in context manager is {round(time.time() - self.init_time, 10)} sec")

In [79]:
with TimerContextManager() as time_cm:
    print(time_cm)
    time.sleep(1)

Hey! I'm context manager!
The time that was spent in context manager is 1.0046401024 sec
